In [ ]:
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from scipy.stats import boxcox

import matplotlib.pyplot as plt

from arch import arch_model

import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE 
from hmmlearn.hmm import GaussianHMM

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# чтение данных Mean monthly air temperature (Deg. F) Nottingham Castle

mean_monthly_temp = pd.read_csv("Series/mean-monthly-air-temperature-deg.csv")

In [ ]:
series = mean_monthly_temp["Deg"]

In [ ]:
# теста Дики-Фуллера

def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

In [ ]:
# Time-series plot

def tsplot(y, lags=None, figsize=(14, 8), style='bmh'):
    test_stationarity(y)
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        plt.figure(figsize=figsize)
        layout = (5, 1)
        ts_ax = plt.subplot2grid(layout, (0, 0), rowspan=2)
        acf_ax = plt.subplot2grid(layout, (2, 0))
        pacf_ax = plt.subplot2grid(layout, (3, 0))
        qq_ax = plt.subplot2grid(layout, (4, 0))

        y.plot(ax=ts_ax, color='blue', label='Or')
        ts_ax.set_title('Original')

        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)
        # qqplot для определения нормальности распределения остатков
        sm.qqplot(y, line='s', ax=qq_ax)
        
        plt.tight_layout()
    return

In [ ]:
# нормализация дисперсии
series = boxcox(series, 0)
# дифференцирование (избавление от тренда)
series = np.diff(series, 1)
# избавление от сезонности
series = series[6:] - series[:-6]
tsplot(series)

In [ ]:
# подбор гиперпараметров для ARIMA

best_aic = np.inf 
best_order = None
best_mdl = None

for i in range(5):
    for d in range(5):
        for j in range(5):
            try:
                tmp_mdl = smt.ARIMA(series, order=(i,d,j)).fit(method='mle', trend='nc')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, d, j)
                    best_mdl = tmp_mdl
            except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))


tsplot(best_mdl.resid, lags=30)

In [ ]:
aic = best_aic
order = best_order 
mdl = best_mdl

In [ ]:
tsplot(mdl.resid)

In [ ]:
# предсказание значений (+ 20 будущих)

with plt.style.context('bmh'):
    plt.figure(figsize=(14,8))
    ax = plt.axes()
    best_mdl.plot_predict(0, len(series)+20, ax=ax)
    plt.plot(series, color='red', label='Series')
    plt.legend()
    plt.show()

In [ ]:
# подгон модели arch, используя наилучшие параметры модели arima
p_ = order[0]
o_ = order[1]
q_ = order[2]

am = arch_model(series, p=p_, o=o_, q=q_, dist='StudentsT')
res = am.fit(update_freq=5, disp='off')
print(res.summary())

In [ ]:
# разбиение данных на train и test

def prepareData(data, lag_start=5, lag_end=20, test_size=0.15):
    
    data = pd.DataFrame(data.copy())
    
    # считаем индекс в датафрейме, после которого начинается тестовый отрезок
    test_index = int(len(data)*(1-test_size))
    
    # добавляем лаги исходного ряда в качестве признаков
    for i in range(lag_start, lag_end):
        data["lag_{}".format(i)] = data.Deg.shift(i)
        
    data = data.dropna()
    data = data.reset_index(drop=True)
    data = data.drop(["Month"], axis=1)
     
    # разбиваем весь датасет на тренировочную и тестовую выборку
    X_train = data.loc[:test_index].drop(["Deg"], axis=1)
    y_train = data.loc[:test_index]["Deg"]
    X_test = data.loc[test_index:].drop(["Deg"], axis=1)
    y_test = data.loc[test_index:]["Deg"]
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = prepareData(mean_monthly_temp, lag_start=1, lag_end=20, test_size=0.3)

In [ ]:
# использоание xgboost для прогнозирования графика

xgb_r = xgb.XGBRegressor(n_estimators = 21) 
  
# Fitting the model 
xgb_r.fit(X_train, y_train) 
  
# Predict the model 
prediction = xgb_r.predict(X_test) 

rmse = MSE(y_test, prediction, squared=False)
print("RMSE : % f" %(rmse)) 

plt.figure(figsize=(14, 8))
plt.plot(prediction, "r", label="prediction")
plt.plot(y_test.values, label="actual")
plt.legend(loc="best")
plt.title("Linear regression")
plt.grid(True);

In [ ]:
data = mean_monthly_temp['Deg']

In [ ]:
values = data.values.reshape(-1,1)

In [ ]:
# прогноз с использованием НММ

model = GaussianHMM(n_components=14,
                        covariance_type="diag",
                        n_iter=1000)
model.fit(values)
labels = model.predict(values)
means = np.zeros_like(values)
for i in range(model.n_components):
    means[labels == i] = model.means_[i]
    
plt.figure(figsize=(12, 6))
plt.plot(values)
plt.plot(means, linewidth=2)

In [ ]:
# прогноз ряда на 20 значений

model = GaussianHMM(n_components=14,
                        covariance_type="diag",
                        n_iter=1000)
model.fit(values[:220])
labels = model.predict(values[221:])
means = np.zeros_like(values[221:])
for i in range(model.n_components):
    means[labels == i] = model.means_[i]
    
plt.figure(figsize=(12, 6))
plt.plot(values)
plt.plot(range(len(values[:221]),len(values)), means,  linewidth=2)
rmse = MSE(values[221:], means, squared=False)
print("RMSE : % f" %(rmse)) 